## 1. Library Imports and Setup
Imports essential libraries for the project including computer vision (cv2), deep learning (torch, YOLO), data processing (numpy), and visualization tools. Sets up logging configuration for tracking the training process.

In [2]:
# Import required libraries
import cv2
import torch
from ultralytics import YOLO
import os
import pickle
import logging
import numpy as np
from datetime import datetime
import colorsys
import shutil
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

# Configure logging
logging.basicConfig(
    filename=f'train_detection_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## 2. COCO Classes and Color Generation
Defines the COCO dataset classes and implements a color generation system that creates distinct colors for visualizing different object classes. Uses HSV color space for better color separation.

In [49]:
# COCO class names (80 classes)
COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog",
    "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork",
    "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
    "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV",
    "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair dryer", "toothbrush"
]

def generate_distinct_colors(num_classes):
    """
    Generate distinct colors for classes using HSV color space.
    Ensures good color separation by varying hue, saturation, and value.
    """
    colors = []
    for i in range(num_classes):
        # Use golden ratio to spread out hues
        hue = (i * 0.618033988749895) % 1.0
        # Full saturation and value for vibrant colors
        saturation = 0.8
        value = 0.9
        
        # Convert HSV to RGB
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        
        # Convert to BGR and scale to 0-255
        bgr_color = (int(rgb[2] * 255), int(rgb[1] * 255), int(rgb[0] * 255))
        colors.append(bgr_color)
    
    return colors

# Generate color map with a color for each class
COLOR_MAP = generate_distinct_colors(len(COCO_CLASSES))

## 3. MOT17 Dataset Preparation
Converts MOT17 dataset into YOLO format. Handles:
- Train/val split (80/20)
- Ground truth conversion
- Image organization
- Label generation in YOLO format
- Dataset configuration (yaml) creation

In [50]:
def prepare_mot17_for_yolo(mot17_root, output_dir):
    """
    Prepare MOT17 dataset for YOLO training by creating a dataset in YOLO format.
    This function converts MOT17 ground truth to YOLO format.
    """
    # Create the necessary directories
    os.makedirs(os.path.join(output_dir, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels', 'val'), exist_ok=True)
    
    # Define the train-val split (80-20)
    sequence_dirs = [d for d in os.listdir(mot17_root) if os.path.isdir(os.path.join(mot17_root, d)) and d.startswith('MOT17-')]
    train_sequences = sequence_dirs[:int(len(sequence_dirs) * 0.8)]
    val_sequences = sequence_dirs[int(len(sequence_dirs) * 0.8):]
    
    logging.info(f"Training sequences: {train_sequences}")
    logging.info(f"Validation sequences: {val_sequences}")
    
    # Process each sequence
    for sequence_dir in sequence_dirs:
        is_train = sequence_dir in train_sequences
        split = 'train' if is_train else 'val'
        
        full_sequence_path = os.path.join(mot17_root, sequence_dir)
        gt_file = os.path.join(full_sequence_path, 'gt', 'gt.txt')
        img_dir = os.path.join(full_sequence_path, 'img1')
        
        # Read ground truth
        if not os.path.exists(gt_file):
            logging.warning(f"Ground truth file not found for {sequence_dir}")
            continue
            
        frame_detections = {}
        with open(gt_file, 'r') as f:
            for line in f:
                data = line.strip().split(',')
                frame_id = int(data[0])
                object_id = int(data[1])
                x, y, w, h = map(float, data[2:6])
                confidence = float(data[6])
                class_id = int(data[7])
                
                # Only consider pedestrians (class 1) with high confidence
                if class_id == 1 and confidence == 1:
                    if frame_id not in frame_detections:
                        frame_detections[frame_id] = []
                        
                    # Store as (x, y, w, h) - will convert to YOLO format later
                    frame_detections[frame_id].append((x, y, w, h))
        
        # Process each frame
        img_files = sorted(glob.glob(os.path.join(img_dir, '*.jpg')))
        for img_file in tqdm(img_files, desc=f"Processing {sequence_dir}"):
            img_path = img_file
            frame_id = int(os.path.basename(img_file).split('.')[0])
            
            # Copy image to dataset directory
            dest_img_path = os.path.join(output_dir, 'images', split, f"{sequence_dir}_{frame_id:06d}.jpg")
            shutil.copy(img_path, dest_img_path)
            
            # Create corresponding label file
            label_path = os.path.join(output_dir, 'labels', split, f"{sequence_dir}_{frame_id:06d}.txt")
            
            # Get image dimensions for normalization
            img = cv2.imread(img_path)
            img_height, img_width = img.shape[:2]
            
            # Convert detections to YOLO format and write to label file
            with open(label_path, 'w') as f:
                if frame_id in frame_detections:
                    for x, y, w, h in frame_detections[frame_id]:
                        # Convert to YOLO format: class x_center y_center width height
                        # All values normalized to [0, 1]
                        x_center = (x + w/2) / img_width
                        y_center = (y + h/2) / img_height
                        width = w / img_width
                        height = h / img_height
                        
                        # In MOT17, pedestrians are class 1, but YOLO expects 0-indexed classes
                        # We'll use "person" which is class 0 in COCO
                        f.write(f"0 {x_center} {y_center} {width} {height}\n")
    
    # Create dataset.yaml file
    with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as f:
        f.write(f"""
path: {output_dir}
train: images/train
val: images/val
nc: 1
names: ['person']
        """)
    
    logging.info(f"MOT17 dataset prepared for YOLO training in {output_dir}")
    return os.path.join(output_dir, 'dataset.yaml')




## 4. Training Callbacks
Implements callbacks for the training process:
- Saves model checkpoints after each epoch
- Records training losses
- Generates loss curve visualizations
- Provides real-time training monitoring

In [51]:
# Define a simple callback function approach
def create_save_epoch_callback(output_dir):
    """Create a callback function for saving epoch checkpoints"""
    losses = []
    epochs_dir = os.path.join(output_dir, "epochs")
    os.makedirs(epochs_dir, exist_ok=True)
    
    def save_epoch_callback(trainer):
        """Method called at the end of each training epoch"""
        # Get current epoch
        epoch = trainer.epoch
        
        # Save the model for this epoch
        epoch_model_path = os.path.join(epochs_dir, f"epoch_{epoch}.pt")
        if hasattr(trainer, 'model'):
            trainer.model.save(epoch_model_path)
        elif hasattr(trainer, 'save'):
            trainer.save(epoch_model_path)
        else:
            # Try a direct save of the trainer object
            torch.save(trainer.model.state_dict(), epoch_model_path)
        
        logging.info(f"Saved model checkpoint for epoch {epoch} to {epoch_model_path}")
        
        # Record loss
        if hasattr(trainer, 'loss') and trainer.loss is not None:
            loss_val = trainer.loss.item() if isinstance(trainer.loss, torch.Tensor) else trainer.loss
            losses.append(loss_val)
    
    def plot_loss_curve():
        if not losses:
            logging.warning("No losses recorded to plot")
            return
                
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(losses) + 1), losses, marker='o')
        plt.title('Training Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)
        
        # Save loss curve image
        loss_curve_path = os.path.join(output_dir, "loss_curve.png")
        plt.savefig(loss_curve_path)
        logging.info(f"Saved loss curve to {loss_curve_path}")
        
        # Show the plot if in interactive environment
        plt.show()
        
        return losses
    
    # Return the callback function and the plotting function
    return save_epoch_callback, plot_loss_curve



## 5. YOLO Training Pipeline
Main training function that:
- Prepares MOT17 dataset
- Loads base YOLOv8 model
- Configures training parameters
- Handles model training
- Saves best weights
- Returns training metrics

In [52]:
def train_yolo_on_mot17(base_model_path, mot17_root, output_dir, epochs=10, batch_size=8, imgsz=640):
    """
    Train a YOLOv8 model on the MOT17 dataset.
    
    Args:
        base_model_path: Path to the base YOLOv8 model to fine-tune
        mot17_root: Path to the MOT17 dataset root directory
        output_dir: Path to save the trained model and dataset
        epochs: Number of training epochs
        batch_size: Batch size for training
        imgsz: Image size for training
    
    Returns:
        Path to the trained model weights and callback object with loss data
    """
    # Prepare the dataset
    yolo_dataset_dir = os.path.join(output_dir, 'yolo_dataset')
    os.makedirs(yolo_dataset_dir, exist_ok=True)
    dataset_yaml = prepare_mot17_for_yolo(mot17_root, yolo_dataset_dir)
    
    logging.info("Starting YOLO training...")
    
    # Load the base model
    model = YOLO(base_model_path)
    
    # Create callback function for saving epoch checkpoints
    save_epoch_callback, plot_loss_curve = create_save_epoch_callback(output_dir)
    
    # Train the model - without specifying callbacks parameter
    try:
        results = model.train(
            data=dataset_yaml,
            epochs=epochs,
            batch=batch_size,
            imgsz=imgsz,
            project=output_dir,
            name="mot17_model",
            exist_ok=True
        )
        
        # Get the path to the best model
        best_model_path = os.path.join(output_dir, "mot17_model", "weights", "best.pt")
        logging.info(f"Training completed. Best model saved to {best_model_path}")
        
        # Plot the loss curve
        losses = plot_loss_curve()
        
        return best_model_path, losses
    
    except Exception as e:
        logging.error(f"Training failed: {e}")
        raise


## 6. Model Loading and Utility Functions
Helper functions for:
- Loading trained YOLO models
- Managing class colors
- Processing individual frames
- Applying confidence thresholds

In [53]:
def load_yolo_model(model_path):
    """Load the YOLOv8 model from the specified path."""
    try:
        model = YOLO(model_path)
        logging.info(f"Successfully loaded YOLOv8 model from {model_path}")
        return model
    except Exception as e:
        logging.error(f"Failed to load YOLO model: {e}")
        raise

def get_class_color(class_id):
    """
    Returns a consistent color for a given class ID.
    Uses modulo to ensure color mapping for any class ID.
    """
    return COLOR_MAP[class_id % len(COLOR_MAP)]

def process_frame(frame, model, frame_id, confidence_threshold=0.5):
    """
    Process a single frame to detect objects and draw labeled bounding boxes with consistent colors.
    Added confidence threshold to filter low-confidence detections.
    """
    detections = []
    try:
        results = model(frame, conf=confidence_threshold)
        logging.info(f"Processed frame {frame_id} with detection results")

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes [x1, y1, x2, y2]
            scores = result.boxes.conf.cpu().numpy()  # Confidence scores
            class_ids = result.boxes.cls.cpu().numpy().astype(int)  # Class IDs as integers

            for box, score, cls in zip(boxes, scores, class_ids):
                # Filter detections based on confidence threshold
                if score >= confidence_threshold:
                    class_name = COCO_CLASSES[cls] if cls < len(COCO_CLASSES) else f"Unknown_{cls}"
                    color = get_class_color(cls)

                    detections.append({
                        "frame": frame_id,
                        "bbox": box.tolist(),
                        "score": float(score),
                        "class_id": int(cls),
                        "class_name": class_name
                    })

                    # Draw bounding box and label
                    x1, y1, x2, y2 = map(int, box)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    label = f"{class_name}: {score:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    except Exception as e:
        logging.error(f"Error processing frame {frame_id}: {e}")
    
    return detections, frame

In [54]:
def detect_objects_in_sequence(model_path, mot17_root, sequence, output_dir, confidence_threshold=0.5):
    """
    Detect objects in an MOT17 sequence, draw labeled bounding boxes, and save results.
    Added optional confidence threshold parameter.
    """
    model = load_yolo_model(model_path)
    video_path = os.path.join(mot17_root, sequence, "img1")

    if not os.path.exists(video_path):
        logging.error(f"Video path {video_path} does not exist")
        raise FileNotFoundError(f"Sequence directory {video_path} not found")

    os.makedirs(output_dir, exist_ok=True)
    output_video_path = os.path.join(output_dir, f"{sequence}_detections.avi")

    all_detections = []
    frame_id = 1
    frame_size = None
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    video_writer = None

    for img_file in sorted(os.listdir(video_path)):
        if img_file.endswith(".jpg"):
            frame_path = os.path.join(video_path, img_file)
            frame = cv2.imread(frame_path)
            if frame is None:
                logging.warning(f"Failed to load frame {frame_path}")
                continue

            if frame_size is None:
                frame_size = (frame.shape[1], frame.shape[0])
                video_writer = cv2.VideoWriter(output_video_path, fourcc, 20.0, frame_size)

            frame_detections, processed_frame = process_frame(
                frame, model, frame_id, confidence_threshold
            )
            all_detections.extend(frame_detections)
            video_writer.write(processed_frame)
            frame_id += 1

    video_writer.release()
    output_file = os.path.join(output_dir, "detections.pkl")
    
    try:
        with open(output_file, "wb") as f:
            pickle.dump(all_detections, f)
        logging.info(f"Saved {len(all_detections)} detections to {output_file}")
    except Exception as e:
        logging.error(f"Failed to save detections: {e}")
        raise

    print(f"Detections completed for {frame_id-1} frames. Results saved to {output_file} and {output_video_path}")

In [55]:
# Set your paths here
base_model_path = r"/csehome/b22ai025/Project/Scratch/yolov8n.pt"  # Base YOLOv8 model
mot17_root = r"/scratch/b22ai025/MOT17/train"      # MOT17 dataset
output_dir = r"/scratch/b22ai025/Results"          # Output directory
# Training parameters
train_epochs = 20
batch_size = 16
image_size = 640

# Detection parameters
sequence_to_evaluate = "MOT17-02-DPM"
confidence_threshold = 0.5

# Step 1: Train the model on MOT17
print("Starting YOLOv8 training on MOT17 dataset...")
trained_model_path, losses = train_yolo_on_mot17(
    base_model_path, 
    mot17_root, 
    output_dir, 
    epochs=train_epochs, 
    batch_size=batch_size, 
    imgsz=image_size
)
print(f"Training completed. Model saved to {trained_model_path}")

# Plot the loss curve
print("Loss data collected during training:", losses)
print("Loss curve was already generated during training")

# Step 2: Run detection on a sequence using the trained model
print(f"Running detection on sequence {sequence_to_evaluate} with trained model...")
detection_output_dir = os.path.join(output_dir, "Detections")
detect_objects_in_sequence(
    trained_model_path, 
    mot17_root, 
    sequence_to_evaluate, 
    detection_output_dir, 
    confidence_threshold
)

print("Training and detection pipeline completed!")

Starting YOLOv8 training on MOT17 dataset...


Processing MOT17-04-SDP: 100%|██████████| 1050/1050 [00:12<00:00, 83.58it/s]


Ultralytics 8.3.98 🚀 Python-3.12.9 torch-2.6.0+cpu CPU (Intel Xeon Gold 6326 2.90GHz)
engine/trainer: task=detect, mode=train, model=/csehome/b22ai025/Project/Scratch/yolov8n.pt, data=/scratch/b22ai025/Results/yolo_dataset/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/scratch/b22ai025/Results, name=mot17_model, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

100%|██████████| 755k/755k [00:00<00:00, 6.42MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      20/20         0G     0.8816     0.4907      0.869         49        640: 100%|██████████| 453/453 [10:59<00:00,  1.46s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:38<00:00,  1.31it/s]

                   all       1617      28327      0.902      0.799      0.904      0.708

20 epochs completed in 4.012 hours.


Optimizer stripped from /scratch/b22ai025/Results/mot17_model/weights/last.pt, 6.2MB
Optimizer stripped from /scratch/b22ai025/Results/mot17_model/weights/best.pt, 6.2MB

Validating /scratch/b22ai025/Results/mot17_model/weights/best.pt...
Ultralytics 8.3.98 🚀 Python-3.12.9 torch-2.6.0+cpu CPU (Intel Xeon Gold 6326 2.90GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:36<00:00,  1.41it/s]


                   all       1617      28327      0.902      0.798      0.904      0.708
Speed: 0.3ms preprocess, 11.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /scratch/b22ai025/Results/mot17_model
Training completed. Model saved to /scratch/b22ai025/Results/mot17_model/weights/best.pt
Loss data collected during training: None
Loss curve was already generated during training
Running detection on sequence MOT17-02-DPM with trained model...

0: 384x640 18 persons, 42.3ms
Speed: 2.2ms preprocess, 42.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 27.0ms
Speed: 1.6ms preprocess, 27.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 28.3ms
Speed: 1.6ms preprocess, 28.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 25.9ms
Speed: 1.6ms preprocess, 25.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 p

## 7. Detection Pipeline
Implements detection on sequences:
- Processes video frames
- Applies object detection
- Draws bounding boxes
- Saves detection results
- Generates visualization videos

In [ ]:
def extract_frames_from_video(video_path, temp_dir):
    """Extract frames from video file and save to temporary directory"""
    os.makedirs(temp_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    if not cap.isOpened():
        raise ValueError(f"Failed to open video file: {video_path}")
    
    frame_paths = []
    print("\nExtracting frames from video...")
    with tqdm(total=total_frames, desc="Frames", bar_format='{l_bar}{bar:30}{r_bar}') as pbar:
        frame_idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            frame_path = os.path.join(temp_dir, f"frame_{frame_idx:06d}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_paths.append(frame_path)
            frame_idx += 1
            pbar.update(1)
    
    cap.release()
    return frame_paths, fps

def generate_detections(model_path, input_path, output_dir, conf_thresh=0.5):
    """Generate detections with controlled output"""
    # Load YOLO model and configure
    model = YOLO(model_path)
    model.conf = conf_thresh
    model.verbose = False
    os.makedirs(output_dir, exist_ok=True)
    
    # Determine input type and get frames
    is_video = input_path.endswith(('.mp4', '.avi', '.mov'))
    if is_video:
        temp_dir = os.path.join(output_dir, 'temp_frames')
        img_files, fps = extract_frames_from_video(input_path, temp_dir)
        total_frames = len(img_files)
        frame_dir = temp_dir
    else:
        frame_dir = os.path.join(input_path, 'img1')
        if not os.path.exists(frame_dir):
            raise ValueError(f"Invalid sequence directory: {input_path}")
        img_files = sorted(os.listdir(frame_dir))
        total_frames = len(img_files)
        fps = 30  # Default for MOT17
    
    # Initialize video writer
    first_frame = cv2.imread(os.path.join(frame_dir, img_files[0]) if not is_video else img_files[0])
    frame_size = (first_frame.shape[1], first_frame.shape[0])
    video_path = os.path.join(output_dir, 'detections.avi')
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), fps, frame_size)
    
    # Process frames
    all_detections = []
    total_dets = 0
    
    print("\nStarting detection pipeline...")
    print("Sample detections (first 5 frames):")
    with tqdm(total=total_frames, desc="Processing", bar_format='{l_bar}{bar:30}{r_bar}') as pbar:
        for frame_id, img_file in enumerate(img_files):
            frame_path = os.path.join(frame_dir, img_file) if not is_video else img_file
            frame = cv2.imread(frame_path)
            results = model(frame)[0]
            frame_dets = 0
            
            # Process detections
            for det in results.boxes.data:
                x1, y1, x2, y2, conf, cls = det
                if int(cls) == 0 and conf > conf_thresh:  # Person class
                    detection = {
                        'frame': frame_id + 1,
                        'bbox': [float(x1), float(y1), float(x2), float(y2)],
                        'score': float(conf)
                    }
                    all_detections.append(detection)
                    frame_dets += 1
                    
                    # Visualize detection
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f'person: {conf:.2f}', 
                              (int(x1), int(y1)-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            out.write(frame)
            total_dets += frame_dets
            
            # Show only first 5 frames details
            if frame_id < 5:
                print(f"Frame {frame_id+1}: {frame_dets} detections")
            elif frame_id == 5:
                print("\n... Processing remaining frames ...\n")
                
            pbar.set_postfix({'dets': total_dets}, refresh=True)
            pbar.update(1)
    
    # Cleanup
    out.release()
    if is_video and os.path.exists(temp_dir):
        import shutil
        shutil.rmtree(temp_dir)
    
    # Save detections
    detections_path = os.path.join(output_dir, 'detections.pkl')
    with open(detections_path, 'wb') as f:
        pickle.dump(all_detections, f)
    
    # Print summary
    print("\n" + "="*50)
    print("Detection Pipeline Summary")
    print("="*50)
    print(f"Input type: {'Video' if is_video else 'MOT17 Sequence'}")
    print(f"Total frames: {total_frames}")
    print(f"Total detections: {total_dets}")
    print(f"Average detections per frame: {total_dets/total_frames:.1f}")
    print(f"\nOutputs saved to:")
    print(f"- Detections: {detections_path}")
    print(f"- Video: {video_path}")
    print("="*50)
    
    return all_detections



In [29]:
def main():
    """Main function with clean output"""
    print("\n" + "="*50)
    print("Object Detection Pipeline")
    print("="*50)
    
    # Get model path
    model_path = "/csehome/b22ai025/Project/Scratch/best.pt"
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found at {model_path}")
    
    # Select input type
    print("\nSelect input type:")
    print("1. MOT17 Sequence")
    print("2. Video file")
    
    while True:
        choice = input("\nEnter choice (1 or 2): ").strip()
        if choice in ['1', '2']:
            break
        print("Invalid choice. Please enter 1 or 2.")
    
    # Handle input based on choice
    if choice == '1':
        sequence_path = input("\nEnter MOT17 sequence path (or press Enter for default): ").strip()
        if not sequence_path:
            sequence_path = "/scratch/b22ai025/MOT17/train/MOT17-02-DPM"
        
        if not os.path.exists(os.path.join(sequence_path, 'img1')):
            raise ValueError(f"Invalid sequence path: {sequence_path}")
            
        output_dir = os.path.join("/csehome/b22ai025/Project/Scratch/Detections", "sequence_output")
        input_path = sequence_path
        
    else:
        video_path = input("\nEnter video file path (.mp4/.avi): ").strip()
        if not video_path.endswith(('.mp4', '.avi', '.mov')):
            raise ValueError("Invalid video format. Supported formats: .mp4, .avi, .mov")
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
            
        output_dir = os.path.join("/csehome/b22ai025/Project/Scratch/Detections", "video_output")
        input_path = video_path
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"\nOutput will be saved to: {output_dir}")
    
    # Run detection
    try:
        generate_detections(model_path, input_path, output_dir)
        print("\nPipeline completed successfully!")
    except Exception as e:
        print(f"\nError during detection: {str(e)}")
        raise

if __name__ == "__main__":
    main()


Object Detection Pipeline

Select input type:
1. MOT17 Sequence
2. Video file

Output will be saved to: /csehome/b22ai025/Project/Scratch/Detections/sequence_output

Starting detection pipeline...
Sample detections (first 5 frames):


Processing:   0%|                              | 0/600 [00:00<?, ?it/s]


0: 384x640 21 persons, 40.1ms
Speed: 2.5ms preprocess, 40.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)


Processing:   0%|                              | 1/600 [00:00<01:24,  7.09it/s, dets=20]

Frame 1: 20 detections

0: 384x640 21 persons, 38.8ms
Speed: 2.5ms preprocess, 38.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 21 persons, 24.2ms
Speed: 1.5ms preprocess, 24.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


Processing: 100%|█████████████████████████████▉| 599/600 [00:34<00:00, 19.48it/s, dets=12712]


0: 384x640 22 persons, 25.2ms
Speed: 1.5ms preprocess, 25.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


Processing: 100%|██████████████████████████████| 600/600 [00:34<00:00, 17.31it/s, dets=12725]


Detection Pipeline Summary
Input type: MOT17 Sequence
Total frames: 600
Total detections: 12725
Average detections per frame: 21.2

Outputs saved to:
- Detections: /csehome/b22ai025/Project/Scratch/Detections/sequence_output/detections.pkl
- Video: /csehome/b22ai025/Project/Scratch/Detections/sequence_output/detections.avi

Pipeline completed successfully!


## 8. DeepSORT Implementation
Custom DeepSORT tracker with:
- Kalman filtering
- Hungarian matching
- Appearance features
- Track management
- ID assignment

In [30]:
# Import required libraries
import cv2
import torch
from ultralytics import YOLO
import os
import pickle
import logging
import numpy as np
from datetime import datetime
import colorsys
import shutil
import glob
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from filterpy.kalman import KalmanFilter

In [31]:
def initialize_kalman_filter():
    """Initialize a Kalman Filter for tracking bounding box motion."""
    kf = KalmanFilter(dim_x=8, dim_z=4)  # State: [x, y, w, h, vx, vy, vw, vh], Measurement: [x, y, w, h]
    
    # State transition matrix (constant velocity model)
    kf.F = np.array([[1, 0, 0, 0, 1, 0, 0, 0],
                     [0, 1, 0, 0, 0, 1, 0, 0],
                     [0, 0, 1, 0, 0, 0, 1, 0],
                     [0, 0, 0, 1, 0, 0, 0, 1],
                     [0, 0, 0, 0, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1]])
    
    # Measurement function (observe only position and size)
    kf.H = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0, 0]])
    
    # Noise covariance matrices
    kf.R = np.eye(4) * 5  # Measurement noise
    kf.P *= 1000.0  # Initial uncertainty
    kf.Q = np.eye(8) * 0.1  # Process noise
    return kf

In [32]:
class Track:
    """Class to manage the state of a single tracked object."""
    def __init__(self, track_id, bbox, appearance):
        # Ensure bbox is a numpy array
        bbox = np.array(bbox, dtype=np.float32)
        
        # Validate bbox shape
        if bbox.size != 4:
            raise ValueError(f"Invalid bbox shape. Expected 4 elements, got {bbox.shape}")
        
        self.track_id = track_id
        self.kf = initialize_kalman_filter()
        
        # Reshape to column vector if needed
        self.kf.x[:4] = bbox.reshape(4, 1)  
        
        self.appearance = appearance  # Appearance descriptor
        self.time_since_update = 0  
        self.hits = 1  
        logging.info(f"Initialized track {track_id} with bbox {bbox}")

    def predict(self):
        """Predict the next state using the Kalman Filter."""
        self.kf.predict()
        self.time_since_update += 1
        return self.kf.x[:4].flatten()

    def update(self, bbox, appearance):
        """Update the track with a new detection."""
        # Ensure bbox is a numpy array
        bbox = np.array(bbox, dtype=np.float32)
        
        if bbox.size != 4:
            logging.warning(f"Invalid bbox for track {self.track_id}. Skipping update.")
            return
        
        self.kf.update(bbox.reshape(4, 1))
        self.appearance = appearance
        self.time_since_update = 0
        self.hits += 1
        logging.debug(f"Updated track {self.track_id} with bbox {bbox}")




In [33]:
class DeepSORT:
    """DeepSORT tracker implementation with improved error handling."""
    def __init__(self, max_age=30, n_init=3, nn_budget=100, 
                 iou_threshold=0.3, appearance_threshold=0.95):
        self.max_age = max_age
        self.n_init = n_init
        self.nn_budget = nn_budget
        self.iou_threshold = iou_threshold
        self.appearance_threshold = appearance_threshold
        self.tracks = []
        self.next_id = 1
        logging.info("Initialized DeepSORT tracker")

    def update(self, detections, frame):
        """
        Update the tracker with new detections for the current frame.
        
        Args:
            detections (list): List of detection dictionaries 
                               with 'bbox' and optional 'appearance' keys
            frame (numpy.ndarray): Current video frame
        
        Returns:
            list: Confirmed tracks with (track_id, bbox)
        """
        # Validate inputs
        if not detections:
            logging.info("No detections in current frame")
            return []
        
        try:
            # Extract bounding boxes
            det_boxes = [
                det.get('bbox') if isinstance(det, dict) else det 
                for det in detections
            ]
            
            # Validate detection boxes
            det_boxes = [
                np.array(box, dtype=np.float32).flatten()[:4] 
                for box in det_boxes if box is not None
            ]
            
            # Compute appearances (histogram-based)
            det_appearances = [
                cv2.calcHist(
                    [frame[int(box[1]):int(box[3]), int(box[0]):int(box[2])]], 
                    [0], None, [256], [0, 256]
                ) for box in det_boxes
            ]

            # Predict existing tracks
            predicted_boxes = [track.predict() for track in self.tracks]

            # Associate detections to tracks
            matches, unmatched_dets, unmatched_tracks = self.associate_detections_to_tracks(
                predicted_boxes, det_boxes, det_appearances
            )

            # Update matched tracks
            for track_idx, det_idx in matches:
                self.tracks[track_idx].update(det_boxes[det_idx], det_appearances[det_idx])

            # Create new tracks for unmatched detections
            for det_idx in unmatched_dets:
                try:
                    new_track = Track(
                        self.next_id, 
                        det_boxes[det_idx], 
                        det_appearances[det_idx]
                    )
                    self.tracks.append(new_track)
                    self.next_id += 1
                except ValueError as e:
                    logging.error(f"Failed to create track: {e}")

            # Remove old unmatched tracks
            self.tracks = [
                t for t in self.tracks 
                if t.time_since_update <= self.max_age
            ]

            # Return confirmed tracks
            confirmed_tracks = [
                (t.track_id, t.kf.x[:4].flatten()) 
                for t in self.tracks 
                if t.hits >= self.n_init
            ]

            logging.debug(f"Active tracks: {len(self.tracks)}")
            return confirmed_tracks

        except Exception as e:
            logging.error(f"Error in tracking update: {e}")
            return []

    def compute_iou(self, box1, box2):
        """Compute Intersection over Union between two bounding boxes."""
        x1, y1, x2, y2 = box1
        x1_, y1_, x2_, y2_ = box2
        xi1, yi1 = max(x1, x1_), max(y1, y1_)
        xi2, yi2 = min(x2, x2_), min(y2, y2_)
        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
        box1_area = (x2 - x1) * (y2 - y1)
        box2_area = (x2_ - x1_) * (y2_ - y1_)
        union_area = box1_area + box2_area - inter_area
        iou = inter_area / union_area if union_area > 0 else 0
        return iou

    def compute_appearance_distance(self, track_appearance, det_appearance):
        """Compute cosine distance between appearance features."""
        try:
            distance = cdist(track_appearance.reshape(1, -1), det_appearance.reshape(1, -1), metric='cosine')[0, 0]
            return distance
        except Exception as e:
            logging.error(f"Error computing appearance distance: {e}")
            return 1.0  # Default to max distance on error

    def associate_detections_to_tracks(self, predicted_boxes, det_boxes, det_appearances):
        """Associate detections to tracks using motion and appearance costs."""
        if not predicted_boxes or not det_boxes:
            return [], set(range(len(det_boxes))), set(range(len(self.tracks)))

        # Compute motion cost (1 - IoU)
        motion_cost = np.array([[1 - self.compute_iou(p, d) for d in det_boxes] for p in predicted_boxes])
        
        # Compute appearance cost
        appearance_cost = np.array([[self.compute_appearance_distance(t.appearance, d) 
                                   for d in det_appearances] for t in self.tracks])
        
        # Combine costs (weighted sum)
        cost_matrix = 0.5 * motion_cost + 0.5 * appearance_cost

        # Hungarian assignment
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        
        # Filter matches based on thresholds
        matches = []
        unmatched_dets = set(range(len(det_boxes)))
        unmatched_tracks = set(range(len(self.tracks)))
        for r, c in zip(row_ind, col_ind):
            if (motion_cost[r, c] < 1 - self.iou_threshold and 
                appearance_cost[r, c] < self.appearance_threshold):
                matches.append((r, c))
                unmatched_dets.discard(c)
                unmatched_tracks.discard(r)

        return matches, unmatched_dets, unmatched_tracks

    def update(self, detections, frame):
        """Update the tracker with new detections for the current frame."""
        # Predict existing tracks
        predicted_boxes = [track.predict() for track in self.tracks]

        # Extract detections
        det_boxes = [det["bbox"] for det in detections]
        det_appearances = [cv2.calcHist([frame[int(box[1]):int(box[3]), int(box[0]):int(box[2])]], 
                                        [0], None, [256], [0, 256]) for box in det_boxes]

        # Associate detections to tracks
        matches, unmatched_dets, unmatched_tracks = self.associate_detections_to_tracks(
            predicted_boxes, det_boxes, det_appearances
        )

        # Update matched tracks
        for track_idx, det_idx in matches:
            self.tracks[track_idx].update(det_boxes[det_idx], det_appearances[det_idx])

        # Create new tracks for unmatched detections
        for det_idx in unmatched_dets:
            new_track = Track(self.next_id, det_boxes[det_idx], det_appearances[det_idx])
            self.tracks.append(new_track)
            self.next_id += 1
            logging.info(f"Created new track {new_track.track_id}")

        # Remove old unmatched tracks
        self.tracks = [t for t in self.tracks if t.time_since_update <= self.max_age]
        logging.debug(f"Active tracks: {len(self.tracks)}")

        # Return confirmed tracks
        confirmed_tracks = [(t.track_id, t.kf.x[:4].flatten()) for t in self.tracks if t.hits >= self.n_init]
        return confirmed_tracks


## 9. MOT Metrics
Evaluation metrics implementation:
- MOTA (Multi-Object Tracking Accuracy)
- MOTP (Multi-Object Tracking Precision)
- IDF1 (ID F1 Score)
- Track statistics

In [34]:
import numpy as np
from collections import defaultdict
import pandas as pd

class MOTMetrics:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.matches = 0
        self.misses = 0
        self.false_positives = 0
        self.id_switches = 0
        self.gt_tracks = set()
        self.pred_tracks = set()
        self.prev_matches = {}
    
    def compute_iou(self, bbox1, bbox2):
        """Compute IOU between two bounding boxes [x,y,w,h]"""
        x1, y1, w1, h1 = bbox1
        x2, y2, w2, h2 = bbox2
        
        # Convert to x1,y1,x2,y2 format
        box1 = [x1, y1, x1+w1, y1+h1]
        box2 = [x2, y2, x2+w2, y2+h2]
        
        xi1 = max(box1[0], box2[0])
        yi1 = max(box1[1], box2[1])
        xi2 = min(box1[2], box2[2])
        yi2 = min(box1[3], box2[3])
        
        intersection = max(0, xi2 - xi1) * max(0, yi2 - yi1)
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = box1_area + box2_area - intersection
        
        return intersection / (union + 1e-6)
    
    def update(self, gt_frame, pred_frame, iou_threshold=0.5):
        """Update metrics for one frame"""
        matched_gt = set()
        matched_pred = set()
        
        # Convert frame data to dictionaries
        gt_dict = {int(row[1]): row[2:6] for row in gt_frame}
        pred_dict = {int(row[1]): row[2:6] for row in pred_frame}
        
        # Update track sets
        self.gt_tracks.update(gt_dict.keys())
        self.pred_tracks.update(pred_dict.keys())
        
        # Compute IOUs between all pairs
        for gt_id, gt_bbox in gt_dict.items():
            for pred_id, pred_bbox in pred_dict.items():
                if pred_id in matched_pred:
                    continue
                    
                iou = self.compute_iou(gt_bbox, pred_bbox)
                if iou >= iou_threshold:
                    matched_gt.add(gt_id)
                    matched_pred.add(pred_id)
                    self.matches += 1
                    
                    # Check for ID switch
                    if gt_id in self.prev_matches and self.prev_matches[gt_id] != pred_id:
                        self.id_switches += 1
                    self.prev_matches[gt_id] = pred_id
                    break
        
        # Count misses and false positives
        self.misses += len(gt_dict) - len(matched_gt)
        self.false_positives += len(pred_dict) - len(matched_pred)
    
    def compute_metrics(self):
        """Compute final MOT metrics"""
        total_gt = self.matches + self.misses
        total_pred = self.matches + self.false_positives
        
        # MOTA (Multi-Object Tracking Accuracy)
        mota = 1 - (self.misses + self.false_positives + self.id_switches) / (total_gt + 1e-6)
        
        # MOTP (Multi-Object Tracking Precision)
        precision = self.matches / (total_pred + 1e-6)
        
        # IDF1 (ID F1 Score)
        idf1 = 2 * self.matches / (total_gt + total_pred + 1e-6)
        
        # Track statistics
        num_tracks = len(self.gt_tracks)
        
        return {
            'MOTA': mota,
            'MOTP': precision,
            'IDF1': idf1,
            'Misses': self.misses,
            'False Positives': self.false_positives,
            'ID Switches': self.id_switches,
            'Total Tracks': num_tracks
        }

## 10. Tracking Pipeline
Complete tracking system that:
- Loads detections
- Applies DeepSORT tracking
- Generates visualizations
- Computes metrics

In [35]:
def evaluate_tracking(gt_path, tracking_results_path):
    """Evaluate tracking results against ground truth"""
    # Load ground truth and tracking results
    gt_data = np.loadtxt(gt_path, delimiter=',')
    pred_data = np.loadtxt(tracking_results_path, delimiter=',')
    
    # Group by frame
    gt_frames = defaultdict(list)
    pred_frames = defaultdict(list)
    
    for row in gt_data:
        if row[6] == 1:  # Consider only ground truth objects marked as visible
            gt_frames[int(row[0])].append(row)
    for row in pred_data:
        pred_frames[int(row[0])].append(row)
    
    # Initialize metrics
    metrics = MOTMetrics()
    
    # Process each frame
    all_frames = sorted(set(gt_frames.keys()) | set(pred_frames.keys()))
    for frame_id in all_frames:
        gt_frame = gt_frames.get(frame_id, [])
        pred_frame = pred_frames.get(frame_id, [])
        metrics.update(gt_frame, pred_frame)
    
    # Compute final metrics
    results = metrics.compute_metrics()
    
    # Print results
    print("\nTracking Evaluation Results:")
    print("-" * 30)
    for metric, value in results.items():
        print(f"{metric}: {value:.3f}")
    
    # Save results to CSV
    df = pd.DataFrame([results])
    results_csv = tracking_results_path.replace('.txt', '_metrics.csv')
    df.to_csv(results_csv, index=False)
    print(f"\nMetrics saved to: {results_csv}")
    
    return results

In [36]:
def plot_metrics(metrics, output_path):
    """Plot tracking metrics visualization"""
    plt.figure(figsize=(12, 6))
    
    # Create bar plot for main metrics
    main_metrics = ['MOTA', 'MOTP', 'IDF1']
    values = [metrics[m] for m in main_metrics]
    
    bars = plt.bar(main_metrics, values, color=['#2ecc71', '#3498db', '#e74c3c'])
    plt.title('Multiple Object Tracking Metrics', fontsize=14, pad=20)
    plt.ylabel('Score', fontsize=12)
    plt.ylim(0, 1)
    
    # Add value labels on bars
    for bar, val in zip(bars, values):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}',
                ha='center', va='bottom')
    
    # Add additional metrics table
    other_metrics = ['Misses', 'False Positives', 'ID Switches']
    other_values = [metrics[m] for m in other_metrics]
    table_data = [[metric, f"{value:.0f}"] for metric, value in zip(other_metrics, other_values)]
    
    plt.table(cellText=table_data,
              cellLoc='center',
              loc='bottom',
              bbox=[0.2, -0.3, 0.6, 0.2])
    
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()

In [37]:
def run_tracking(input_path, detections_pkl, output_path, gt_path=None):
    """Run tracking pipeline for both sequence and video inputs"""
    # Load detections
    with open(detections_pkl, 'rb') as f:
        all_detections = pickle.load(f)
    
    # Initialize tracker
    tracker = DeepSORT(
        max_age=30,
        n_init=3,
        nn_budget=100,
        iou_threshold=0.3,
        appearance_threshold=0.95
    )
    
    # Determine input type
    is_video = input_path.endswith(('.mp4', '.avi', '.mov'))
    
    if is_video:
        # For video input, read frames directly
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise ValueError(f"Failed to open video: {input_path}")
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        ret, first_frame = cap.read()
        if not ret:
            raise ValueError("Failed to read first frame")
        
        frame_size = (first_frame.shape[1], first_frame.shape[0])
    else:
        # For sequence input, read from img1 directory
        img_dir = os.path.join(input_path, 'img1')
        if not os.path.exists(img_dir):
            raise ValueError(f"Invalid sequence directory: {img_dir}")
        
        img_files = sorted(os.listdir(img_dir))
        total_frames = len(img_files)
        fps = 30  # Default for MOT17
        first_frame = cv2.imread(os.path.join(img_dir, img_files[0]))
        frame_size = (first_frame.shape[1], first_frame.shape[0])
    
    # Initialize video writer
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, frame_size)
    
    # Initialize tracking results
    tracking_results = []
    
    # Process frames
    print("\nRunning tracking...")
    with tqdm(total=total_frames, desc="Tracking", bar_format='{l_bar}{bar:30}{r_bar}') as pbar:
        frame_id = 0
        
        while True:
            if is_video:
                ret, frame = cap.read()
                if not ret:
                    break
            else:
                if frame_id >= total_frames:
                    break
                frame = cv2.imread(os.path.join(img_dir, img_files[frame_id]))
            
            # Get detections for current frame
            frame_dets = [d for d in all_detections if d['frame'] == frame_id + 1]
            
            if frame_dets:
                # Update tracker
                tracks = tracker.update(frame_dets, frame)
                
                # Process tracks
                for track_id, bbox in tracks:
                    x1, y1, x2, y2 = bbox
                    w, h = x2 - x1, y2 - y1
                    
                    # Store results
                    tracking_results.append([
                        frame_id + 1, track_id, x1, y1, w, h, 1, -1, -1, -1
                    ])
                    
                    # Visualize
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), 
                                (0, 255, 0), 2)
                    cv2.putText(frame, f'ID: {track_id}', (int(x1), int(y1)-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Write frame
            out.write(frame)
            frame_id += 1
            pbar.update(1)
    
    # Cleanup
    out.release()
    if is_video:
        cap.release()
    
    # Save tracking results
    if tracking_results:
        results_path = output_path.replace('.avi', '.txt')
        np.savetxt(results_path, np.array(tracking_results),
                  delimiter=',',
                  fmt='%d,%d,%.2f,%.2f,%.2f,%.2f,%d,%d,%d,%d')
    
    # Evaluate if ground truth is provided
    if gt_path and os.path.exists(gt_path):
        print("\nEvaluating tracking results...")
        metrics = evaluate_tracking(gt_path, results_path)
        return metrics
    
    return None

In [38]:
def main():
    """Main function for tracking with multi-input support"""
    print("\n" + "="*50)
    print("Multi-Object Tracking Pipeline")
    print("="*50)
    
    # Select input type
    print("\nSelect input type:")
    print("1. MOT17 Sequence")
    print("2. Video file")
    
    while True:
        choice = input("\nEnter choice (1 or 2): ").strip()
        if choice in ['1', '2']:
            break
        print("Invalid choice. Please enter 1 or 2.")
    
    # Handle input based on choice
    if choice == '1':
        # MOT17 Sequence
        sequence_path = input("\nEnter MOT17 sequence path (or press Enter for default): ").strip()
        if not sequence_path:
            sequence_path = "/scratch/b22ai025/MOT17/train/MOT17-02-DPM"
        
        if not os.path.exists(sequence_path):
            raise ValueError(f"Invalid sequence path: {sequence_path}")
            
        input_path = sequence_path
        detections_pkl = os.path.join("/csehome/b22ai025/Project/Scratch/Detections", "sequence_output", "detections.pkl")
        output_path = os.path.join("/csehome/b22ai025/Project/Scratch/Detections", "sequence_output", "tracked.avi")
        gt_path = os.path.join(sequence_path, "gt/gt.txt")
        
    else:
        # Video file
        video_path = input("\nEnter video file path (.mp4/.avi): ").strip()
        if not video_path.endswith(('.mp4', '.avi', '.mov')):
            raise ValueError("Invalid video format. Supported formats: .mp4, .avi, .mov")
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
            
        input_path = video_path
        detections_pkl = os.path.join("/csehome/b22ai025/Project/Scratch/Detections", "video_output", "detections.pkl")
        output_path = os.path.join("/csehome/b22ai025/Project/Scratch/Detections", "video_output", "tracked.avi")
        gt_path = None  # No ground truth for video input
    
    # Create output directory
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Verify detections file exists
    if not os.path.exists(detections_pkl):
        raise FileNotFoundError(
            f"Detections file not found: {detections_pkl}\n"
            "Please run detection pipeline first!"
        )
    
    # Run tracking pipeline
    print("\nStarting tracking pipeline...")
    print(f"Using detections from: {detections_pkl}")
    
    try:
        metrics = run_tracking(input_path, detections_pkl, output_path, gt_path)
        
        print("\n" + "="*50)
        print("Tracking Pipeline Summary")
        print("="*50)
        print(f"Input type: {'MOT17 Sequence' if choice == '1' else 'Video'}")
        print(f"Output saved to: {output_path}")
        
        if metrics:
            print("\nTracking Metrics:")
            print("-"*20)
            for metric, value in metrics.items():
                print(f"{metric}: {value:.3f}")
        else:
            print("\nNote: Evaluation skipped (no ground truth available)")
        
        print("="*50)
        
    except Exception as e:
        print(f"\nError during tracking: {str(e)}")
        raise

if __name__ == "__main__":
    main()


Multi-Object Tracking Pipeline

Select input type:
1. MOT17 Sequence
2. Video file
Invalid choice. Please enter 1 or 2.
Invalid choice. Please enter 1 or 2.
Invalid choice. Please enter 1 or 2.
Invalid choice. Please enter 1 or 2.

Starting tracking pipeline...
Using detections from: /csehome/b22ai025/Project/Scratch/Detections/sequence_output/detections.pkl

Running tracking...


Tracking: 100%|██████████████████████████████| 600/600 [00:21<00:00, 27.74it/s]



Evaluating tracking results...

Tracking Evaluation Results:
------------------------------
MOTA: 0.529
MOTP: 0.783
IDF1: 0.769
Misses: 4529.000
False Positives: 3905.000
ID Switches: 325.000
Total Tracks: 62.000

Metrics saved to: /csehome/b22ai025/Project/Scratch/Detections/sequence_output/tracked_metrics.csv

Tracking Pipeline Summary
Input type: MOT17 Sequence
Output saved to: /csehome/b22ai025/Project/Scratch/Detections/sequence_output/tracked.avi

Tracking Metrics:
--------------------
MOTA: 0.529
MOTP: 0.783
IDF1: 0.769
Misses: 4529.000
False Positives: 3905.000
ID Switches: 325.000
Total Tracks: 62.000
